In [1]:
import sys
import sklearn
import numpy as np
import os
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

In [2]:
import quandl as q 

In [3]:
q.ApiConfig.api_key = '11sy5SMsFERx_YypjsMi'

In [4]:
data = q.get('BCHAIN/MKPRU') 

In [5]:
data

,Value
Date,
2009-01-02,0.00
2009-01-03,0.00
2009-01-04,0.00
2009-01-05,0.00
2009-01-06,0.00
...,...
2021-10-28,58538.49
2021-10-29,60587.09
2021-10-30,62249.18
